In [14]:
# package imports
import base64
from dotenv import load_dotenv
import json
import lyricsgenius as lg
import math
import numpy as np
import os
import pandas as pd
import requests
import urllib

# relative imports
import auth.spotify as spotify

In [5]:
# read environment variables
load_dotenv()
# client id and secret
SPOTIFY_API_CLIENT_ID = os.getenv('SPOTIFY_API_CLIENT_ID')
SPOTIFY_API_CLIENT_SECRET = os.getenv('SPOTIFY_API_CLIENT_SECRET')
# refresh token
SPOTIFY_REFRESH_TOKEN = os.getenv('SPOTIFY_REFRESH_TOKEN')
# api urls
SPOTIFY_API_BASE_URL = os.getenv('SPOTIFY_API_BASE_URL')
SPOTIFY_API_TOKEN_URL = os.getenv('SPOTIFY_API_TOKEN_URL')
# redirect uri
SPOTIFY_API_REDIRECT_URI = os.getenv('SPOTIFY_API_REDIRECT_URI')
# api paths
TRACK_AUDIO_FEATURES_ENDPOINT = os.getenv('TRACK_AUDIO_FEATURES_ENDPOINT')
USER_TOP_TRACKS_ENDPOINT = os.getenv('USER_TOP_TRACKS_ENDPOINT')

In [6]:
res = requests.get(f"{SPOTIFY_API_BASE_URL}{USER_TOP_TRACKS_ENDPOINT}", headers = {
    "Authorization": f"Bearer {spotify.get_access_token()}"
}).json()

# print(json.dumps(res['items'][0], indent=2))

In [7]:
top_tracks = []
for item in res['items']:
    artists = [artist['name'] for artist in item['artists']]
    total_seconds = math.floor(item['duration_ms'] / 1000)
    leftover_seconds = total_seconds % 60
    duration = f"{total_seconds // 60}:{'0{}'.format(leftover_seconds) if leftover_seconds < 10 else leftover_seconds}"
    track = [
        item['id'],
        item['name'],
        ', '.join(artists),
        duration,
        item['album']['id'],
        item['album']['name'],
        item['album']['images'][0]
    ]
    top_tracks.append(track)

In [8]:
def fetch_track_audio_features(track_id):
    res = requests.get(f"{SPOTIFY_API_BASE_URL}{TRACK_AUDIO_FEATURES_ENDPOINT}{track_id}", headers = {
        "Authorization": f"Bearer {spotify.get_access_token()}"
    }).json()
    return res

for track in top_tracks:
    audio_features = fetch_track_audio_features(track[0])
    track += [audio_features['danceability'], audio_features['energy'], audio_features['key'],
              audio_features['loudness'], audio_features['mode'], audio_features['speechiness'],
              audio_features['acousticness'], audio_features['instrumentalness'], audio_features['liveness'],
              audio_features['valence'], audio_features['tempo']]
    

In [15]:
GENIUS_API_BASE_URL = os.getenv('GENIUS_API_BASE_URL')
GENIUS_API_ACCESS_TOKEN = os.getenv('GENIUS_API_ACCESS_TOKEN')

genius = lg.Genius(GENIUS_API_ACCESS_TOKEN)

In [16]:
artist_name = 'Khruangbin'
artist = genius.search_artist(artist_name, max_songs=10)
for song in artist.songs:
    print(f"Title: {song.title}, Lyrics: {song.lyrics[:100]}...")  # Print a portion of the lyrics

# TODO: search for song lyrics (100-500 char should be enough) and append to df

# print(len(top_tracks[0]))
# tracks_df = pd.DataFrame(top_tracks, columns=[
#     'Track ID', 'Track Name', 'Artists', 'Duration', 'Album ID', 'Album Name', 'Image',
#     'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness',
#     'Liveness', 'Valence', 'Tempo'
# ])
# display(tracks_df)

Searching for songs by Khruangbin...

Song 1: "White Gloves"
Song 2: "Cómo Te Quiero"
Song 3: "Pelota"
Song 4: "So We Won’t Forget"
Song 5: "Connaissais de Face"
Song 6: "Time (You and I)"
Song 7: "Friday Morning"
Song 8: "Evan Finds the Third Room"
Song 9: "Lady and Man"
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Khruangbin-como-me-quieres-lyrics
Song 10: "Cómo Me Quieres"

Reached user-specified song limit (10).
Done. Found 10 songs.
Title: White Gloves, Lyrics: 17 ContributorsWhite Gloves Lyrics[Verse 1]
She was a queen
She had a house
She was a fighter
She wa...
Title: Cómo Te Quiero, Lyrics: 10 ContributorsCómo Te Quiero Lyrics[Intro]
(Ooh)
(Ooh)
You're always
Forever in my heart

[Interlud...
Title: Pelota, Lyrics: 14 ContributorsPelota Lyrics[Verse 1]
Soy una pelota
Una pelota de hollín
Yo estaba perdido
En una c...
Title: So We Won’t Forget, Lyrics: 17 ContributorsSo We Won’t Forget Lyrics[Chorus]
Ooh, one to rememb